In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf
from sklearn.utils import shuffle
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split

from tensorflow.keras import models, layers
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
import pathlib
data_dir =pathlib.Path("dataset")

In [3]:
data_dir

WindowsPath('dataset')

In [4]:
retina_images_dict = {
    'mild' : list(data_dir.glob('Mild/*')),
    'moderate' : list(data_dir.glob('Moderate/*')),
    'no_dr' : list(data_dir.glob('No_DR/*')),
    'proliferate_dr' : list(data_dir.glob('Proliferate_DR/*')),
    'severe' : list(data_dir.glob('Severe/*')),
}

In [5]:
retina_labels_dict = {
    'mild' : 0,
    'moderate' : 1,
    'no_dr' : 2,
    'proliferate_dr' : 3,
    'severe' : 4
}

In [6]:
x, y = [], []
for disease_level, images in retina_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        x.append(img)
        y.append(retina_labels_dict[disease_level])

In [7]:
x = np.array(x)
y = np.array(y)

In [8]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=879)

In [9]:
xval, xtest, yval, ytest = train_test_split(xtest, ytest, test_size=0.5, random_state=879)

In [10]:
img_size = 224
b_size = 32
channels = 3
epoch = 2

In [11]:
#scaling
resize_rescale = tf.keras.Sequential([
    #all the image have the same size then why to use??
    #when user input new image of different size then useful
    layers.experimental.preprocessing.Resizing(img_size, img_size),
    layers.experimental.preprocessing.Rescaling(1.0/255),
])

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
import math

In [13]:
i_shape = (b_size, img_size, img_size, channels)
base_densenet_model = Sequential([
    resize_rescale,
    DenseNet201(input_shape=(img_size, img_size, channels), weights="imagenet", include_top=False)
], name="base_densenet_model")

base_densenet_model.trainable = False
base_densenet_model.build(i_shape)
base_densenet_model.summary()


Model: "base_densenet_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 224, 224, 3)         0         
                                                                 
 densenet201 (Functional)    (None, 7, 7, 1920)        18321984  
                                                                 
Total params: 18,321,984
Trainable params: 0
Non-trainable params: 18,321,984
_________________________________________________________________


In [14]:
i_shape = (b_size, img_size, img_size, channels)
print(i_shape)

(32, 224, 224, 3)


In [15]:
custom_densenet_model = Sequential([
    base_densenet_model,
    GlobalAveragePooling2D(),
    Dropout(0.2),
    Dense(units=1, activation="softmax")
], name = "custom_densenet_model")

custom_densenet_model.build(i_shape)

In [16]:
custom_densenet_model.summary()

Model: "custom_densenet_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 base_densenet_model (Sequen  (32, 7, 7, 1920)         18321984  
 tial)                                                           
                                                                 
 global_average_pooling2d (G  (32, 1920)               0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (32, 1920)                0         
                                                                 
 dense (Dense)               (32, 1)                   1921      
                                                                 
Total params: 18,323,905
Trainable params: 1,921
Non-trainable params: 18,321,984
_________________________________________________________________


In [25]:
custom_densenet_model.compile(optimizer='adam',
             loss = tf.keras.losses.BinaryCrossentropy(from_logits=False),
             metrics=['accuracy'])

In [18]:
early = EarlyStopping(monitor="val_loss",
                     patience=math.floor(epoch*0.1))

learning_rate_reduction = ReduceLROnPlateau(monitor="val_loss",
                                           patience=2,
                                           verbose=1,
                                           factor=0.3,
                                           min_lr=0.000001)
modelcheck = ModelCheckpoint('best_model.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
history = custom_densenet_model.fit(xtrain, ytrain, 
          epochs= epoch, batch_size = b_size,
         verbose=1, validation_data=(xval, yval))

Epoch 1/2
740/776 [===========================>..] - ETA: 3:08 - loss: -156.1308 - accuracy: 0.2019